# Combinación de marcos de datos en `Pandas`.

In [1]:
import pandas as pd
import numpy as np

Para poder realizar este ejercicio, se usan los *merge-join* que se hacen habitualmente en SQL.

- **LEFT JOIN**. Conserva únicamente las observaciones de la tabla de la izquierda, pegándole las observaciones de las características de la tabla de la derecha que coincidan con las de la izquierda según la columna de unión.
- **RIGHT JOIN**. Es semejante al **LEFT JOIN**, salvo que la prioridad la tiene la tabla de la derecha.
- **INNER JOIN.** Solo se combinan las observaciones comunes para ambas tablas.
- **OUTER JOIN**. Se combinan todas las tablas independientemente de que tengan elementos comunes o no.

Para poder hacer eso, se puede usar la función `pd.concat()` junto con el parámetro `axis`.

- `axis = 0`. Se usa para pegar una tabla debajo de otra, semejante al `rbind()` de `R`.
- `axis = 1`. Se usa para pegar una tabla a la izquierda o a la derecha de otra, semejante al `cbind()` de `R`.

Para poner el ejemplo, creemos el siguiente marco de datos con Pandas:

In [4]:
datos_1 = pd.DataFrame({
    'A': ['A0', 'A1', 'A2', 'A3'],
    'B': ['B0', 'B1', 'B2', 'B3'],
    'C': ['C1', 'C2', 'C3', 'C4'],
    'D': ['D1', 'D2', 'D3', 'D4']
})

datos_2 = pd.DataFrame({
    'A': ['A4', 'A5', 'A6', 'A7'],
    'B': ['B4', 'B5', 'B6', 'B7'],
    'C': ['C4', 'C5', 'C6', 'C7'],
    'D': ['D4', 'D5', 'D6', 'D7']
})

Vamos a pegar `datos_2` debajo de `datos_1`. Vale la pena anotar que, por defecto, `Python` usa `axis = 0`. Veamos:

In [6]:
pd.concat([datos_1, datos_2])

,A,B,C,D
0,A0,B0,C1,D1
1,A1,B1,C2,D2
2,A2,B2,C3,D3
3,A3,B3,C4,D4
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


¡Y se han pegado una debajo de la otra! No obstante, es está trayando los mismos índices que se tenían originalmente, lo cual es confuso porque se tienen índices que se duplican. Para evitar eso, se puede usar el argumo `ignore_index = True`.

In [8]:
pd.concat([datos_1, datos_2], ignore_index = True)

,A,B,C,D
0,A0,B0,C1,D1
1,A1,B1,C2,D2
2,A2,B2,C3,D3
3,A3,B3,C4,D4
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


Y ahora se resetean los índices, comenzando desde cero e incrementando de uno a uno. Ahora bien, si queremos que una tabla quede a la izquierda o a la derecha de la otra, usamos `axis = 1`.

In [9]:
pd.concat([datos_1, datos_2], axis = 1)

,A,B,C,D,A,B,C,D
0,A0,B0,C1,D1,A4,B4,C4,D4
1,A1,B1,C2,D2,A5,B5,C5,D5
2,A2,B2,C3,D3,A6,B6,C6,D6
3,A3,B3,C4,D4,A7,B7,C7,D7


### Función `merge`.

In [12]:
datos_3 = pd.DataFrame({
    'key': ['k0', 'k1', 'k2', 'k3'],
    'A': ['A0', 'A1', 'A2', 'A3'],
    'B': ['B0', 'B1', 'B2', 'B3']
})

datos_4 = pd.DataFrame({
    'key': ['k0', 'k1', 'k2', 'k3'],
    'C': ['C0', 'C1', 'C2', 'C3'],
    'D': ['D0', 'D1', 'D2', 'D3']
})

Supongamos que queremos juntar a las tablas `datos_3` y `datos_4` usando como columna de unión a `key`.

In [14]:
datos_3.merge(datos_4, on = 'key')

,key,A,B,C,D
0,k0,A0,B0,C0,D0
1,k1,A1,B1,C1,D1
2,k2,A2,B2,C2,D2
3,k3,A3,B3,C3,D3


Si las llaves tienen nombres distintos:

In [26]:
datos_5 = pd.DataFrame({
    'llave': ['k0', 'k1', 'k2', 'k3'],
    'A': ['A0', 'A1', 'A2', 'A3'],
    'B': ['B0', 'B1', 'B2', 'B3']
})

datos_6 = pd.DataFrame({
    'pk': ['k0', 'k1', 'k2', 'k3'],
    'C': ['C0', 'C1', 'C2', 'C3'],
    'D': ['D0', 'D1', 'D2', 'D3']
})

In [20]:
datos_5.merge(datos_6, left_on = 'llave', right_on = 'pk')

,llave,A,B,pk,C,D
0,k0,A0,B0,k0,C0,D0
1,k1,A1,B1,k1,C1,D1
2,k2,A2,B2,k2,C2,D2
3,k3,A3,B3,k3,C3,D3


¡Y se pegan! Sin embargo, me trae las dos llaves, lo cual puede ser resultante. Esto se puede resolver fácilmente usando un `drop`.

In [21]:
datos_5.merge(datos_6, left_on = 'llave', right_on = 'pk').drop(['pk'], axis = 1)

,llave,A,B,C,D
0,k0,A0,B0,C0,D0
1,k1,A1,B1,C1,D1
2,k2,A2,B2,C2,D2
3,k3,A3,B3,C3,D3


# `Join`.

Se usa para apuntarle a filas específicas y no a columnas. Para ello realiza un *index match*.

In [27]:
datos_5.set_index('llave', inplace = True)
datos_6.set_index('pk', inplace = True)

Para poder juntar estos dos marcos de datos usando los índices:

In [28]:
datos_5.join(datos_6)

,A,B,C,D
llave,,,,
k0,A0,B0,C0,D0
k1,A1,B1,C1,D1
k2,A2,B2,C2,D2
k3,A3,B3,C3,D3
